In [ ]:
import pandas as pd
from gc import collect as gc_collect

In [ ]:

# Extraction des SIRET des DECP
decp_sirets_df = pd.read_csv("dist/decp.csv", dtype="object", index_col=None, 
                      usecols=['titulaire.id', 'titulaire.typeIdentifiant'])

decp_sirets_df = decp_sirets_df[decp_sirets_df['titulaire.typeIdentifiant'] == 'SIRET']
decp_sirets_df = decp_sirets_df.drop_duplicates()
decp_sirets_df.index.size

In [ ]:

# Récupération des données SIRET
dtypes_etablissements={
    'siret': 'object',
    'siren': 'object',
    'longitude': 'float',
    'latitude': 'float',
    'activitePrincipaleEtablissement': 'object',
    'codeCommuneEtablissement': 'object'
}
etablissement_df_chunked = pd.read_csv('data/StockEtablissement_utf8_geo.csv.gz', chunksize=(500000), dtype=dtypes_etablissements, index_col=None, 
                               usecols=['siret', 'siren', 'longitude', 'latitude', 'activitePrincipaleEtablissement', 'codeCommuneEtablissement'])

merged_chunks_list = []

# 6 min 20 en gardant tous les merges, 10 000 chunks. Resultat 38M rows!
# 6 min 27 avec inner merge et en ne gardant que les merge non vides
# 4 min 04 avec 200 000 chunks
# 3 min 27 avec 500 000 chunks
# 3 min 05 avec 1M chunks

with etablissement_df_chunked as reader:
    for df_chunk in reader:
        merge = pd.merge(decp_sirets_df, df_chunk, how='inner', left_on='titulaire.id', right_on='siret', )
        if merge.index.size > 0:
            merged_chunks_list.append(merge)

# TODO: attention, je dois faire gaffe à distinger df des titulaires avec sirets et df avec toutes les données DECP

decp_sirets_df = pd.concat(merged_chunks_list).drop(columns=['siret'])
decp_sirets_df

In [ ]:

# Récupération des données SIREN
# Possibilité d'utiliser les URL sources en prod au lieu d'un fichier local

dtypes_uniteLegales = {
    'siren': 'object',
    'categorieEntreprise': 'category',
    'etatAdministratifUniteLegale': 'category',
    'economieSocialeSolidaireUniteLegale': 'category',
    'categorieJuridiqueUniteLegale': 'category'
}
unite_legale_df_chunked = pd.read_csv('./data/StockUniteLegale_utf8.zip', index_col=None, dtype=dtypes_uniteLegales, sep=",", chunksize=500000,
                              usecols=['siren', 'categorieEntreprise', 'etatAdministratifUniteLegale',
                                      'economieSocialeSolidaireUniteLegale', 'categorieJuridiqueUniteLegale'])


with unite_legale_df_chunked as reader:
    for df_chunk in reader:
        merge = pd.merge(decp_sirets_df, df_chunk, how='inner', left_on='siren', right_on='siren')
        if merge.index.size > 0:
            merged_chunks_list.append(merge)
del unite_legale_df_chunked

decp_sirets_df = pd.concat(merged_chunks_list).drop(columns=['siret'])


decp_sirets_df = decp_sirets_df.drop(columns=['siren'])



In [ ]:


# Raccourcissement du code commune
decp_sirets_df['departement'] = decp_sirets_df['codeCommuneEtablissement'].str[:2]
decp_sirets_df = decp_sirets_df.drop(columns=['codeCommuneEtablissement'])

# Raccourcissement de l'activité principale
# TODO: à vérifier
decp_sirets_df['activitePrincipaleEtablissement'] = decp_sirets_df['activitePrincipaleEtablissement'].str[:-3]

# Correction des données ESS et état
decp_sirets_df['etatAdministratifUniteLegale'] = decp_sirets_df['etatAdministratifUniteLegale'].replace({'A': 'Active', 'C': 'Cessée'})
decp_sirets_df['economieSocialeSolidaireUniteLegale'] = decp_sirets_df['economieSocialeSolidaireUniteLegale'].replace({'O': 'Oui', 'N': 'Non'})


In [ ]:

# Récupération et raccourcissement des categories juridiques du fichier SIREN
# TODO: à vérifier le [:-3]
decp_sirets_df['categorieJuridiqueUniteLegale'] = decp_sirets_df['categorieJuridiqueUniteLegale'].astype(str).str[:-3]

# Récupération des libellés des catégories juridiques
cj_df = pd.read_csv('cj.csv', index_col=None, dtype="object")
categorieJuridiqueUniteLegale = pd.merge(decp_sirets_df, cj_df, how='left', left_on='categorieJuridiqueUniteLegale', right_on='code')


In [ ]:
# Jointure des DECP et des données SIRENE
decp_selection_df = pd.read_csv('decp-selection.csv')
siren_siret_df = pd.read_csv('siren-siret.csv')
decp_titulaires_df = pd.merge(decp_selection_df, siren_siret_df, how='left', left_on='titulaire.id', right_on='siret')
decp_titulaires_df = decp_titulaires_df[['id', 'uid', 'acheteur.id', 'acheteur.nom', 'nature', 'objet', 'codeCPV', 'lieuExecution.code', 'lieuExecution.typeCode', 'lieuExecution.nom', 'dureeMois', 'dateNotification', 'montant', 'titulaire.id', 'titulaire.typeIdentifiant', 'titulaire.denominationSociale', 'codeAPE', 'departement', 'categorie', 'categorieJuridique', 'categorieJuridiqueLibelle1', 'categorieJuridiqueLibelle2', 'etatEtablissement', 'etatAdministratifUniteLegale', 'longitude', 'latitude', 'donneesActuelles', 'anomalies']]
decp_titulaires_df.to_csv('decp-titulaires.csv', index=False)
